*****************************************************
### The sentiment of sentence.
### Stanford nlp

In [1]:
from pandas import Series, DataFrame
import pandas as pd
from numpy.random import randn
import numpy as np
import re
from pycorenlp import StanfordCoreNLP

In [2]:
clean_score7_hs_df = pd.read_csv("./Data/clean_hs_df_7.csv", sep='\t', encoding="utf-8").drop("Unnamed: 0", axis=1).set_index(['home_id'])
clean_score8_hs_df = pd.read_csv("./Data/clean_hs_df_8.csv", sep='\t', encoding="utf-8").drop("Unnamed: 0", axis=1).set_index(['home_id'])
clean_score9_hs_df = pd.read_csv("./Data/clean_hs_df_9.csv", sep='\t', encoding="utf-8").drop("Unnamed: 0", axis=1).set_index(['home_id'])
clean_score10_hs_df = pd.read_csv("./Data/clean_hs_df_10.csv", sep='\t', encoding="utf-8").drop("Unnamed: 0", axis=1).set_index(['home_id'])
loc_score7_hs_df = pd.read_csv("./Data/loc_hs_df_7.csv", sep='\t', encoding="utf-8").drop("Unnamed: 0", axis=1).set_index(['home_id'])
loc_score8_hs_df = pd.read_csv("./Data/loc_hs_df_8.csv", sep='\t', encoding="utf-8").drop("Unnamed: 0", axis=1).set_index(['home_id'])
loc_score9_hs_df = pd.read_csv("./Data/loc_hs_df_9.csv", sep='\t', encoding="utf-8").drop("Unnamed: 0", axis=1).set_index(['home_id'])
loc_score10_hs_df = pd.read_csv("./Data/loc_hs_df_10.csv", sep='\t', encoding="utf-8").drop("Unnamed: 0", axis=1).set_index(['home_id'])

In [3]:
print(len(clean_score7_hs_df))
print(len(clean_score8_hs_df))
print(len(clean_score9_hs_df))
print(len(clean_score10_hs_df))
print(len(loc_score7_hs_df))
print(len(loc_score8_hs_df))
print(len(loc_score9_hs_df))
print(len(loc_score10_hs_df))

80
75
75
75
28
128
75
75


In [4]:
nlp = StanfordCoreNLP('http://localhost:9000')

def stf_nlp_sentiment(text):
    
    if not text:
        return None
    else:
        res = nlp.annotate(text,
                           properties={
                               'annotators': 'sentiment',
                               'outputFormat': 'json'
                           }) 
        return res["sentences"][0]["sentimentValue"]

def sentiment_summary(hs_df):

    sentiment_summary_df = pd.DataFrame(index = hs_df.index, 
                                        columns=['num_of_sents_0.6',
#                                                  'num_very_neg_0.5', 'num_neg_0.5', 'num_neut_0.5', 'num_pos_0.5', 'num_very_pos_0.5',
                                                 'num_very_neg_0.6', 'num_neg_0.6', 'num_neut_0.6', 'num_pos_0.6', 'num_very_pos_0.6',
                                                 'num_sents'
#                                                  'num_very_neg_0.7', 'num_neg_0.7', 'num_neut_0.7', 'num_pos_0.7', 'num_very_pos_0.7' 
                                                ])
    for idx in hs_df.index:
#         sentiment_summary_df.loc[idx][:6] = count_sentiment(hs_df, idx, 'sents_0.5')
        sentiment_summary_df.loc[idx] = count_sentiment(hs_df, idx, 'sents_0.6')
#         sentiment_summary_df.loc[idx][6:11] = count_sentiment(hs_df, idx, 'sents_0.6')
#         sentiment_summary_df.loc[idx][-5:] = count_sentiment(hs_df, idx, 'sents_0.7')
    
    return sentiment_summary_df

def count_sentiment(hs_df, idx, col):
    
    num_very_neg, num_neg, num_neut, num_pos, num_very_pos = 0, 0, 0, 0, 0
    
    for s in hs_df.loc[idx][col]:
        print(s)
        sentiment_value = stf_nlp_sentiment(s)
        
        if sentiment_value:
            if sentiment_value == '1':
                num_neg += 1
            elif sentiment_value == '2':
                num_neut += 1
            elif sentiment_value == '3':
                num_pos += 1
            elif sentiment_value== '4':
                num_very_pos += 1
            else:
                num_very_neg += 1
        else:
            continue
            
    return [hs_df.loc[idx]['num_of_sents_0.6'], num_very_neg, num_neg, num_neut, num_pos, num_very_pos, hs_df.loc[idx]['num_of_sents']]

#     if col.endswith('5'):
#         return [hs_df.loc[idx]['num_of_sents'],  num_very_neg, num_neg, num_neut, num_pos, num_very_pos]
#     else:
#         return [num_very_neg, num_neg, num_neut, num_pos, num_very_pos]

def sim_str_to_list(hs_df, col):
    for idx in hs_df.index:
        string = hs_df.loc[idx][col].strip('[').strip(']')
        sim_string = re.split('\', |\", ', string)
        hs_df.at[idx, col] = [s.strip('\ \'').strip('\"') for s in sim_string]

def get_sentiment_summary(hs_df):
#     sim_str_to_list(hs_df, 'sents_0.5')
    sim_str_to_list(hs_df, 'sents_0.6')
#     sim_str_to_list(hs_df, 'sents_0.7')
    return sentiment_summary(hs_df)

In [5]:
stf_nlp_sentiment("Not a huge deal, but when I arrived the woman cleaning the home was still there and didn't leave until about 3:30.")

'1'

In [6]:
stf_nlp_sentiment("The place does need a bit of a clean up.")

'3'

In [7]:
stf_nlp_sentiment("what a horrible clean.")

'1'

In [8]:
import os
import json

# Functions to get the API predicted results
def getLabel(text):
    cmd = 'curl -d "text=' + text + '" http://text-processing.com/api/sentiment/'
    res = os.popen(cmd).read()
    return json.loads(res)['label']

In [9]:
getLabel("what a horrible clean.")

'neg'

In [10]:
clean_score7_sents_sentiment_df = get_sentiment_summary(clean_score7_hs_df)
clean_score7_sents_sentiment_df

Neat, tidy, authentic.
However, if you are an individual who is used to a very clean living space - like myself -
I am constantly traveling for work and look forward to a clean, comfortable night away from hotels when I utilize airbnb, which has happened in past stays.  \r\n\r\n
A comfortable bed, full access to the kitchen and living room,  he even cleaned out a space for me in his garage for my motorcycle .
If you are looking for a sparkling-clean place with a newer kitchen and bath, you may be disappointed.
But the bed was decent, and overall, we found it to be quiet, tidy, and cozy.
By the time we arrived, the place was ready, spotlessly clean and bare.
Cleanliness was an issue for me, however.
A bit dirty.
I'll probably spend a bit more money next time to hopefully get a cleaner place.
The place was unclean.
The bathtub was dirty, had signs of mold, and had half-used bars of soap lying in it.
On the stove top there were 2 unclean looking pots and there were tiny ants scurrying thr

The apartment is very pleasant, clean and comfortable.
There are a few pots & pans but we had to ask for a coffee maker;\n- there are no waste baskets, just a garbage container in the kitchen;\n- we had 3 towels for 3 of us & had to ask for wash cloths and hand towels but never got the hand towels.
were very dirty;\n-there is a view
When I finally reached his wife, she first said she didn't know we had a reservation, then said we were expected at 2:00 or 3:00, then told us we couldn't get in for a couple of hours because the apartment was being cleaned!
She was there, gave us the keys but said we couldn't move in until the cleaning was done.
The 4-story building is functional, but not particularly attractive or clean.
Cleanliness is an issue.
There is ingrained dirt and stains on walls and carpets in the building and even in the apartment, that makes one question whether it is real dirt or the remnants of past dirt.
And it makes one question the cleanliness of sheets and towels, though

The microwave and oven was very dirty, so couldn't even use it.
It was clean and the projection screen with Netflix helped when there was a rainy night night and we had to stay in.
The building is older, but the apartment was clean and well equipped.
Everything was clean and well kept.
The place needed a professional cleaning.
The rug was quite messes and smelly.
The linens looked spot cleaned vice washed.
Clean, a bit aged/dated as described, but very comfortable.\r\n\r\n
The space was clean, bright, and comfortable.
Apartment was comfortable and clean.
Eric had clean trash cans and had a new roll of toilet paper ready for us apron arrival.
The only two downfalls of the space was that the kitchen could use some updating (dirty and not many utensils ) and that the TV was super laggy.
Place was nice and clean and very welcoming.
1.Anna is still new to the AirBNB hosting process and is in a learning curve as to having a system of how clean a rented room and bathroom should be.
Within  30

,num_of_sents_0.6,num_very_neg_0.6,num_neg_0.6,num_neut_0.6,num_pos_0.6,num_very_pos_0.6,num_sents
home_id,,,,,,,
30712,43,0,24,9,9,1,452
299817,0,0,0,0,0,0,19
613151,1,1,0,0,0,0,13
639130,0,0,0,0,0,0,31
666897,8,0,6,0,2,0,97
670009,3,0,1,1,1,0,87
670021,5,1,1,1,2,0,74
716829,0,0,0,0,0,0,7
1145590,1,0,0,0,1,0,46


In [11]:
clean_score8_sents_sentiment_df = get_sentiment_summary(clean_score8_hs_df)
clean_score8_sents_sentiment_df

What a neat place with a lovely backyard - great location, great host, great pets!A friendly place.
Sweet pets, very clean and nice.
sunsets- stunning views only a few blocks away!This room was in a great location and the room was clean and comfortable.
A little disappointed with the cleanliness of the house and room (the room smelled like cats and mildew).
The bed is very nice, the bathroom clean, the kitchen fully equipped.
You'll be missing out terribly if all you're looking for is a clean and tidy place to crash.
Don't hesitate to book his clean, quaint and well maintained (and sign posted) private room!
The apartment was clean and quiet.
Great host, comfy, clean room with all you need in the most buzzing part of Seattle.
Overall, incredibly good value and highly recommended!!The apartment was clean and really nice.
Thanks!Nick does his best to provide the things you would need for a comfortable stay (clean sheets, towels, plates, wifi!!).
Even though its at the back of the buildin

The apartment itself was tidy and even though there was a heat wave and no A/C, it was kept cool by being on the bottom floor and the host provided several fans to chill the space.
Great location and clean.
On your way up to the door you'll see tons of spiderwebs and dust.
Place was neat
Great location, clean apartment, simple entry process.
This accommodation was easily accessible and we arrived to a spotlessly clean apartment.
The property was clean and comfortable.
It was clean overall, but dusty in areas--couch was old & worn, and I couldn't reach the outlet next to the bed (lamp was plugged in but stereo/alarm was unplugged).
The apartment is clean and presents as advertised.
The apartment itself was very clean and nicely furnished.
Clean linens and towels were provided.
The space was very clean and it felt like my own apartment.
his place was clean and comfortable with all amenities provided.
His place is little off from the downtown, where touring area is, but it cost me only $1

The house was clean and spacious.
The only words of caution are it’s not for small kids (not safety proofed) and not for fastidious folks (bring your own sheets and wash clothes if you are used to the Ritz).
The place was spacious, clean and in a really fun area of Seattle.
I would recommend this place for large groups and people who just want to hang out and party in a neat neighborhood.
Not a huge deal, but when I arrived the woman cleaning the home was still there and didn't leave until about 3:30.
We were also supposed to have until 4pm to check out, but on the day of our reservation Farhad emailed me to say he forgot there was another group arriving that night and would need more time to be able to clean before they arrived, and wanted to know if we could leave early.
Overall, we were satisfied--and especially pleased that the host provided a means to recycle and compost.
Although clean, this felt like a party house where I lived in college.
It was fascinating and disgusting.
(Tho

She did offer to let us store our bags in the apartment while it was being cleaned but consistent communication would have been helpful.
The room was nice enough but located right near a trash room so very smelly when you exit the room.
My feet were covered in black dirt from walking on the floors bear foot despite washing them constantly so the floors could use some extra cleaning.
There were only two towels for two people and no wash cloths.
One of the towels was still wet when we arrived.
Inside was clean and nice.
The apartment is old-ish but clean and apartment building looks like some social housing...
Thanks for the stay!The place was clean, quaint and cozy, and it has everything you could need.
Everything was clean and I felt welcomed.
Also we were curios if the house was been cleaned or not because there were some spots on the bed cover as well as dust in between the bed and the wall.
We arrived to a leftover trashbag and dirty dishes, but Nicole was quick to apologize and han

The only downsides, which I suspect Owen would take care of if I cared enough about these factors to text him, were the lack of a hand towel in the bathroom, the sink in the kitchen needed a scrubbing, and there were a few dust bunnies all around (if you have allergies).
The apartment is a decent size, but could have used a little bit more cleaning; nothing terrible, but some relatively minor things that could be taken care of pretty easily.  \r\n\r\n
The apartment was clean, and the view was perfect as I sat at the desk getting last minute work finished.
The apartment was clean well maintained.
Place was good, clean and location wise convenient.
Neat apartment in a great location.
-)The apartment is cozy and clean.
Very clean, great location, walking to downtown, and simply pleasant to hang out in.
The room was quiet and clean, as advertised.
- it was basic, in a great location, and clean.
and rather clean.
The room is as described, clean with a bed and private entry.
Facilities are c

The room was very clean and looked exactly like it did in the picture.
I had to ask for clean sheets.
We were offered no discount or compensation for the bugs; we were even charged a “cleaning fee.
The room was lovely and sunny and the rest of the house was clean.
The room was comfortable and clean.
The house is very nice, clean and tidy, with a nice little garden plot out front.
The room itself was clean and comfortable, and it was a little cool the first night but Reena provided a nice thick quilt that made the rest of the nights even better.
I should also mention that the areas weren't dirty, per se, but the bathroom had (water?)
The place was clean and comfortable, and the neighborhood was beautiful, with large houses and nice landscaping, and right next to a Discovery Park with trails etc.
Pros: \r\n- lots of character \r\n- very clean\r\n- GORGEOUS view \r\n- easy access to the freeway\r\n- lock box for getting/depositing keys (quick and simple!)\r\n\r\n
We found the apartment to

both Paul and Andy patiently heard my feedback and seemed amenable to improving the cleanliness situation.\r\n\r\n
It's a busy house (but still kept perfectly clean) with plenty of other interesting guests, so the place has a nice social vibe.
The house was quiet and kept clean and everyone I encountered was friendly.
All the house is clean and good environment!!!
This large old home – clean but worn

Their place in Ballard is comfy, clean and modern and her family were very welcoming and very lovely! \r\n\r\n
They didn't wash their dishes and damp the gabage bin  in time as well in the kitchen.
The room is clean and the location is close to lots of fun restaurants and shops in the Ballard area.
The room was very clean.
We were only looking for a place to sleep for a cheap price in a clean place - this is exactly what we found!
Our room was clean and quaint.
He didn't want to mess with them, so he had to go somewhere else - but obviously this is not a big deal, just something to commun

I am personally allergic to fabric softners of the bedding, so i had to sleep with my own towel covering the pillow
Unfortunately,kitchen and bath are not very clean.
The queen size mattress is the most comfortable one I ever experienced on AirBnB. But to be honest, I didn't recommend so much if you seek for a clean living environment.
The kitchen is shared with 3 other room guests so it is just a big mess when you entered the kitchen.
And also the floor is not clean very much.
Bed was comfy and clean.
The room was clean, quiet, and was in a good neighborhood.
The bathroom is a bit messy but the living room, kitchen and dinning room are clean.
Very clean and tidy, close to downtown Seattle which was great! \r\n
The air mattress was quite comfortable but the apartment could have been cleaner.
She made me feel welcomed and already had the air mattress  out and a clean towel out for me
Good location but the appartement could have been à bit cleaner.
We booked the room last minute, and Ale

,num_of_sents_0.6,num_very_neg_0.6,num_neg_0.6,num_neut_0.6,num_pos_0.6,num_very_pos_0.6,num_sents
home_id,,,,,,,
15108,7,0,3,0,3,1,189
258571,103,0,36,12,52,3,1406
264829,15,0,3,2,8,2,216
278192,5,0,0,1,4,0,68
293890,12,0,4,2,6,0,177
338091,28,1,13,1,11,2,374
606297,28,0,12,5,10,1,497
637710,24,0,11,6,7,0,285
719233,127,2,41,26,52,6,1510


In [12]:
clean_score9_sents_sentiment_df = get_sentiment_summary(clean_score9_hs_df)
clean_score9_sents_sentiment_df

The place was spotless and very homey.
This is a big house and the cleaning staff did a great job of making the place sparking clean for us.
Most of our questions were answered immediately and everything was neatly put down in the handy house manual for quick reference.
Location is perfect, the house is super clean, the beds are so comfy (which is a sad but true rarity for most airbnbs
When I arrived this this trip the hot tub hot recently been cleaning and drained.
The host cleaned the house and left the lights on for us, it was walking distance to the Convention Center and Capital Hill restaurants and pubs, the shower is amazing.
The house was clean and big.
The fees that Air bnb charges was not clearly stated and the cleaning fee wasn't included in the price of the apt.
The room was comfortable, clean and well-decorated.
The room is very clean and has almost all of the amenities that you'll need.
Great location for walking around the city, and nice, clean accommodations.
The apartme

The studio was cozy, clean, and conveniently located near bars, cafes, markets, as well as public transportation.
Check-in and check-out were super easy and the place is quiet and clean.
The studio was great--clean, comfortable and had all the essentials, and was in a lovely neighborhood with a short walk to fun, eclectic Capitol Hill.
Everything was clean and comfortable.
The apartment was clean and cozy and all the necessities were provided.
We found the place to be clean and spacious and the WiFi works well.
Super clean, comfortable, cute and relatively economical.
If you clean up after yourself you can save yourself a cleaning fee which is fantastic.
Everything was very neat and clean, the appliances were working, and I loved the artistic touches throughout the house.
Place is in a convenient location off street, so no traffic noise and very clean.
The room was clean and spacious.
The room was small but tidy and comfortable.
It is a great room with a desk, very clean kitchen, clean

The building has a lot of character, is clean, spacious, and has the friendliest people running it.
The unit was very nice, clean and had everything that we needed.
Just be sure the apartment you want is the apartment you get when booking and you will have a great stay!Clean and tidy apartment and a great host.
While the unit itself is very clean, the building could use some work and the lower floors reek of old cigarette smoke.
Clean, well stocked apartment with bathroom (tp, shampoo, soap, towels, sheets and pillows and blankets) and kitchen essentials (cookware, seasoning, dish towels) and fans and air conditioning.
Wash and dry are $1 each a few floors down in a dedicated washer room. \n\n
The unit I got was an alternate unit, but still bright, clean and spacious.
The place was clean and cozy!
It was clean, the bed was comfy, and had good water pressure.
The condo was very neat and orderly.
The condo itself is clean, spacious and warmly inviting.
The place is secure and clean, havi


it was tidy and very clean.
The room was clean, tidy and private - everything you want!
The bedroom and sitting room were clean and roomy, as was the private bathroom.
The room is very clean, comfortable, house has all the necessary amenities.
The room was clean and inviting.
The room is very clean.
The room and bathroom were clean, well stocked and both the bed and the couch were very comfortable.
The room was spacious and very clean.
The bed was comfortable, the bathroom was clean and right off our room, and it was quiet so we got plenty of sleep.
The room/bathroom were clean.
She was pleasant and welcoming and the bedroom, bathroom and common areas were clean and well kept.
Betty was a wonderful host and her place was lovely, clean, and comfortable!
Our room was lovely-clean, quiet, and comfortable and had everything we needed.
Enjoyed our stay!Quiet street, clean room.
We were very comfortable and the house is clean and lovely.
The room was nice and clean, it was convenient to  ta

Her apartment was tidy and was quite accommodating.
Overall, I wild highly recommend this place for ease of check-in/out, location, cleanliness, host response time, and uniqueness.
The apartment room is clean and spacious and the apartment building itself is charming with a lot of character.
The space was clean and just as pictured!
The space was clean, exactly as depicted, and in a great location.
I especially appreciated the clean sheets, towels and bath products without scents and chemicals, and the bins for recycling and compost.
Our other bigger gripe just had to do with cleanliness and the layout of the bathroom...
so it's also entirely wet afterwards.
First the private room & shower; very easy to get to, very neat and clean, and SUPER comfortable bed.
The room was clean and cozy and perfect for the two of us (especially after a long day of exploring Seattle)!
The apartment was clean and spacious.
The apartment was clean, modern, spacious and conveniently located.
The apartment i

The place was clean and was easy to find.
The place was very clean, comfy, and spacious.
The apartment was clean and the photos shown were accurate.
Everything was clean, the bed was comfy, and the shower was hot.
The place was very clean and cozy, spa like atmosphere with many amenities.
The home was well kept and clean, and had parking as well.
Lauren is a awesome host and her place is very nice and clean.
The bed was comfortorbale, the pool was big, and the apt was very clean.
impeccably clean and well appointed.
Lauren was very organized and welcoming, and the apartment is very nice and clean.
The apartment is really nice and clean.
The place was nice and clean, no complaints!Great space and location!
Very clean, bright and modern home in the heart of Capitol Hill.
To find a kitchen off limits due to the smell emanating from a sink full of dirty dishes and uneaten leftovers, an overflowing trash bin and multiple days of dirty pots and pans still on the stovetop from the previous fe

Was informed that the apartment had been "professionally cleaned" but the bedsheets had many hairs on them and the bed didn\'t have a bottom (fitted) sheet.
The apartment has a very modern feel, clean and without clutter.
Reginald was extremely accommodating with my check-in time and the place was extremely clean and well appointed.
Comfortable and clean.
Apartment was very clean and had everything we needed.
The apartment itself was nice and neat.
Everything was clean and easy to find.
Also, the apartment is clean and comfortable.
Reginald's place is very cool: Convenient location, clean apartment, and gracious host!

The room is spacious, clean.
Location is great and her place was clean and comfortable.
The home was cozy,  clean, and I loved the scented candles!
The room is clean, comfortable and quite spacious, and the common areas are very well kept.
Clean, fresh bed and bedding, cool lighting in the room, wifi, toiletries. \r\n\r\n
Very nice and clean basement apartment with frien

,num_of_sents_0.6,num_very_neg_0.6,num_neg_0.6,num_neut_0.6,num_pos_0.6,num_very_pos_0.6,num_sents
home_id,,,,,,,
13068,7,0,4,1,1,1,144
19611,13,0,7,0,5,1,167
86185,6,1,1,0,3,1,199
132120,10,0,1,0,9,0,127
286712,29,1,10,0,17,1,244
442487,43,0,9,4,25,5,577
444221,30,0,12,0,16,2,303
458186,5,0,1,0,4,0,79
479653,22,0,3,1,16,2,344


In [13]:
clean_score10_sents_sentiment_df = get_sentiment_summary(clean_score10_hs_df)
clean_score10_sents_sentiment_df

The house itself is spacious and clean, and Judith was very open to us sharing all of the common space on the first floor (kitchen, dining, living room).
The guest suite has a comfortable bed, a huge bathroom (both super clean).
Comfy bed, clean and roomy accommodations
The home was well maintain and clean with wonderful views easy access to food and entertainment.
The house itself was clean, well-maintained, and comfortable.
The  home is beautiful and our was very cosy and clean.
Everything was placed neatly to accommodate anything a  traveler/visitor could want.
The room and attached bath are on the main floor, everything was very clean and comfortable.
Everything is clean, comfortable, convenient, and the views of the sound in the morning (especially on sunny days) just make you feel happy all over!
The house has spectacular views and it is a very comfortable and clean place to stay.
The house was very architecturally unique, beautifully decorated, and the rooms were very clean.
Jud

The Redstone apartment was great - clean and spacious.
The pictures Jessie posts about her place don't even do it justice: so cozy, cute, clean, feels like home, and in a great area.
My personal recommendations: don't miss to try Red Mill Burger, go to Greelake and if you like wine try Conde Valdemar (Spanish wine, you can buy it in the market near the house - Ken's Market).Jessie's apartment is aesthetically pleasing, very clean and very close to all of the lively amenities of Ballard and Phinney Ridge.
Super clean, super comfortable. \r\n
Although we were only there to sleep mostly, it was clean and very cozy.
The place is comfortable, relaxing, clean, and splendidly fitted with funky decor art and features that make you feel right at home.
The apartment was also very clean, and very recently remodeled, with many well-chosen, high-quality materials.
Not only was it great to have a "home base" ready for us as soon as we arrived in town, but we found the apt to be beautiful, clean, and

Clean, well-equipped and very comfortable!Great location!
Very clean and cute apartment, will definitely stay again.
It was clean and cozy, in a wonderful neighborhood, and close to downtown Seattle.

Queen Anne is so central and the vibe was clean and safe.
A sincere thank you to Kelly for a most enjoyable Seattle experience!Kelly was a great host and this is a clean, well-equipped, private apt near downtown.
Her place was immaculately clean, wifi was good, shower was fantastic, and the bed was comfy.
Her apartment is immaculate and extremely clean.
Very clean and comfortable, the coffee, bear and chocolate were a treat too, thank you!
Even better than advertised!Kelly's place is very nice and clean and it look exactly like the pictures.
The house is clean and simple - we had all we need during our stay there :)
Her home is lovely, warming, very clean and equipped with everything a great home needs.
It's super clean.
It is also VERY clean, which we appreciated.
We had everything we ne

The house was clean and bright, the neighborhood walkable with things to do nearby, and only minutes from downtown.
We had a great time at the carriage house - very clean, fantastic design and surprisingly comfortable!
The craftsmanship of the work is impeccable and the place is immaculately clean.
The space was clean and minimalistic, and the tall ceilings made the carriage house seem so big and open.
Altogether, this place was very warm, cozy and clean.
Very clean and open feel.
The space was very clean, well designed, and quiet.
They had a folder full of smartly-written recommendations and information, and Seattle guidebooks--
The accommodations are sparkling clean and well designed and the location is great.
The house was very well kept and tidy; exactly as the photos portray.
The loft is very clean, spacious with really cool modern interior design.
Everything was exactly as pictured and super clean.
The house was very clean and put together, making us feel comfortable and relaxed.

The futon mattress in the room is great (especially if you prefer a hard mattress) and comes with clean sheets.
The room was clean and well appointed, and the bed was comfortable.
The room was clean and neat and the bed was very comfortable.
The room/house/bathroom were all very cozy and clean.
The place was CLEAN!
Clean, spacious, and very convenient location.
The home was spotless as I walked in.
The clothes washer and dryer have a large capacity, and both complete a load on regular cotton settings in under an hour.
Apartment is very clean and beautiful.
Everything was clean and dust free.
It was clean and spacious.
His house, the bedroom, and bathroom were all clean and comfortable.
The house is extremely clean!
The place is very clean and has all you could need.
Nice, very clean bedroom and bathroom.
The space was better than advertised, clean, comfortable, had unexpected extras (large flat screen TV, full pot of coffee in the am), and in a great location for visiting SPU. \r\n
Tha

The house is modern-designed,  specious,clean,neat and tidy.
The place is very homey and clean and comfortable.
The house was clean, spacious and he left instructions on everything as well as some tourist advise and recommendations.
The place was spacious, clean and well organized.
Freshly backed cookies, clean house, lightly stocked refrigerator,  and a personalized note addressed to our family was our welcome to Keith's house.
He was easy to get in touch with, the house was very clean, and he left freshly baked cookies.
Home is exactly as described, was clean, bright, well stocked, VERY comfortable beds, FANTASTIC location within easy walking distance to GreenLake, Phinney Ridge, bus stop, easy car parking out front, thoroughly enjoyed our stay.

Great location, very clean and the bed is extremely comfortable.
Smartly furnished, very clean and orderly, and sun-drenched.
I felt very cozy because Susan prepared everything I need: clean towel, hairdryer, shampoo, electric kettle, heat a

The room was clean and quiet and Jon was a very friendly host.
The place was impeccably clean. \r\n\r\n
Thanks Jon!Good location, clean & comfy digs.
The spot itself was clean and very welcoming.
❤ we have never stayed anywhere so great through air bnb, house was safe, easy access and so clean!
We love it that Jon provides a clean, comfortable place to relax.
The place is very clean and well decorated with more amenities than I could ask for such as an accessible computer, Netflix, and board games.
The apartment was bigger than expected and impeccably clean.
I wish this was my apartment!Jarad's place was exactly as described, clean and what I wanted in a property.
Would definitely stay again!Jared was a great host and his apartment was very clean and in a great location, walking distance from downtown.
The place is extremely clean and has air condition.
This place was clean and perfectly spacious, and the minimalist design made for a very relaxing, hotel-like stay (nothing better than 

However, bedroom floors were dirty...
Dusty and lots of hair.. and bathroom would've been better if it were cleaner.
The place is cozy, clean and private.
clean and comfy!
The room is really nice, and the house is clean and tidy.
The room was exactly as described and clean and ready for me.
Airbnb place including kitchen and bathroom is really nice and clean.

The space was clean, hip and well stocked for a stay (hairdryer, iron, well outfitted big kitchen, nice towels,etc.).
It was super clean, had everything available that I needed, and was nice to relax in between adventures.\r\n\r\n
the space was perfect and really clean, had a great view of the city and the bed was super comfortable.
The place is very clean, efficient, and there are lots of details she has covered.
It has a great view and was spotless.
The house was comfortable, clean and warm.

Clean, strong wifi, and positive vibe.
Everything was clean and there were freshly washed towels waiting for me.
The apartment is incredi

,num_of_sents_0.6,num_very_neg_0.6,num_neg_0.6,num_neut_0.6,num_pos_0.6,num_very_pos_0.6,num_sents
home_id,,,,,,,
117165,20,0,2,0,11,7,584
182716,16,0,1,0,12,3,346
215882,56,0,7,1,41,7,964
320704,29,0,7,1,20,1,447
394408,9,0,0,3,6,0,207
555182,2,0,0,0,1,1,76
825574,18,0,4,0,12,2,238
1159226,6,0,0,0,3,3,72
1198799,10,0,1,1,5,3,93


In [14]:
loc_score7_sents_sentiment_df = get_sentiment_summary(loc_score7_hs_df)
loc_score7_sents_sentiment_df

The neighborhood was a little scary.
The house was nice, and the neighborhood wasn't bad either.
Probably one of the less seedy neighborhoods I saw while in Seattle.
The neighborhood feels relatively safe.
it's not suburbia, but sometimes you have to sacrifice that when in the city for convenience and closeness to all the activities you want to do.
Thanks Tony!Great location.
Safe neighborhood.
Definitely a little bit on the lower income and not as gentrified as closer in, thank goodness!
Centrally located, we could have use public transportation or drive definitely
The bus station to downtown is directly in on the other side of the street, as well as a nice cafe.
Next door is a nice bar, and a few blocks downtown you find some nice bars and restaurants.
The area is not the best if you read or listen to others, but we didńt have any trouble.
If you go down the streets, about 10 min. walk to Columbia city, you will find plenty of places to eat, drink or shop. \r\n
The flat was charming 

The neighborhood is described as an "up and coming" neighborhood which means there is a range of nice and rundown houses.
The location was very good for us, within walking distance of family.
The neighborhood is run-down, but we felt safe.
The home was located at a small street in a older neighborhood that has seen better times.
We arrived in the evening, and it was a little difficult navigating through dark streets.
However, during our stay, we didn't experience any issues with the neighbors.\r\n\r\n
Two out of the three room's window/bug screen were missing, and the screens in the dinning area + washroom had holes in it.
It was great value for the distance from downtown Seattle considering the general cost of the area.
There is a train station only a short walk from her house which goes downtown.\r\n\r\n
It was a little bit of a walk to the train with a lot of luggage, but besides that it was in a nice location close to the city centre.
As someone who lived in the Seattle for a littl

Je give as advice an toked as to the gas station that was near by.\n
The apartment is walking distance to downtown.
Tania father is very knowledgable about the area and gave me some great advice.



,num_of_sents_0.6,num_very_neg_0.6,num_neg_0.6,num_neut_0.6,num_pos_0.6,num_very_pos_0.6,num_sents
home_id,,,,,,,
23356,9,0,6,1,2,0,35
190984,57,3,25,4,24,1,299
613151,0,0,0,0,0,0,13
1022135,5,0,2,1,2,0,41
1263470,10,0,2,3,5,0,67
1549973,6,0,5,0,1,0,43
3706719,4,0,0,0,3,1,15
4041868,10,0,3,4,3,0,47
4126284,8,0,3,2,2,1,40


In [15]:
loc_score8_sents_sentiment_df = get_sentiment_summary(loc_score8_hs_df)
loc_score8_sents_sentiment_df

The place is really close to downtown and in a very central location.
There is a parking spot right in front of the location and if you take public transit the bus and light rail stations are close.
Keith also operates car & shuttle services so he can pick you up and drop you off at the airport and many other locations.
Nice neighborhood to walk my dog around
His place is little off from the downtown, where touring area is, but it cost me only $15 including the tip for the drivier to get to the downtown where SAM is.:)The place was great, clean and comfy.
downtown.
His place is not far from downtown and pretty easy access to I-5 and 90.
It also had a great location,  It was easy to get to downtown, capital hill,,the U district and to the eastside,  yet it was in a nice, quiet and safe neighborhood.
The location was great and I hope to stay again.
Very close to downtown.
Also, the neighborhood was quiet.
Driving to downtown was not far at all.
Keith & Cherie also offered us rides to dow

The location is nice because it is short 10 minute drive from downtown, and the house is very close to the light rail station, which you can take directly downtown in 15 minutes or to the airport.
The Beacon Hill area is a pretty typical Seattle neighborhood--nothing swanky but perfectly nice.
For our group it worked very well because it was a very economical option near downtown with plenty of space, and the fact that Prez and Cherie were so responsive and interested in making sure we had a good time was a nice perk.  \r\n\r\n
We had a great weekend with our college friends in Seattle, everyone in our group was pleased with the accommodations, and we would recommend it to anyone looking for a good value in a location but who does not need a nice (but not fancy) place.
The house was very clean and fit our needs, it was located close to the Train station so heading
downtown was a sinch and very quick.
We didn't get to meet in person since I was too busy with my brother's wedding nearby 

Theres a grocery store nearby.
The neighborhood looked a little concerning when we first pulled in but seemed to be fine.
15 mins from downtown.
It was close to the light rail, which made getting to and from downtown Seattle super easy.
The house was located in a nice, safe neighborhood.
It was easy to park and located about a 10 minute drive to SafeCo Field.
The house was in a good location and they had zero complaints.
the location is also great.
Great place in a quiet location, prefect if you are going for work as I was.
Qing was very helpful during my stay and I would definitely stay here againCharming cottage, centrally located.
Onsite parking was very helpful because of two hour parking restriction in the neighborhood.
She gave us some suggestions for restaurants nearby, and then gave us our privacy.
The neighbourhood was nice and quiet and we had easy access to the city.
The area is quiet and centrally located.
This cottage was just the right size and the right location....
enou

It is close to downtown and to all the places we needed to get to in Seattle.
It is a bit north of downtown and is not a great neighborhood for walking, but it is a perfect home base to see the city from.
The location is very close to the highway and plenty of restaurant.
Very quiet location despite being so close to Aurora/99.
Very easy access to downtown (straight shot down Aurora/99 takes about 15 minutes - except in rush hour).
The location is very strategic.
Location was great, it was in a very safe neighborhood near downtown.
The apartment has a nice location.
We drove to the Green Lake nearby nearly everyday.
Also there are two supermarkets nearby.
And not to far away from downtown.
Neighborhood is great.
Within one block from E Line, which takes you downtown & within walking distance to local neighborhood restaurants & breweries.
Decent place to stay while in Seattle, this location is centrally located.
very cozy, very close to the E bus route which takes you straight to downto

It is very easy to get to downtown by bus.
Their home is clean, comfy and in a great location for exploring the city (gotta love the Aurora express bus!)
The bedroom space is perfectly private, yet a comfortable community feel when you open the door to the dining area and the bathroom.
There really isn't much to do in the neighborhood and eating options are slim to none in the neighborhood.
Located near everything you need and Tahoma, Laura and Spencer couldn't been more welcoming and accommodating.
The room was comfortable, the description was accurate and the neighbourhood is very nice.
The connections to downtown Seattle are very easy to use and conveniently located.
There are a shower and also a small kitchen and dinning area. \r\n
You would share the basement area (kitchen and shower) with Spencer who is super friendly guy. \r\n
The connection at kitchen area is great, but not in the room.
I didn't hire a car during this trip, and the location of this house made it very easy for m

...

From the selection of beds to the warmth of the home and the location, we loved every aspect of our stay.
While the neighborhood may at first appear to be ho hum and maybe not the trendiest, it was quiet and a perfect distance from both the airport (about 15 minutes) and downtown (again... 15 minutes).
The proximity to highways and such was key.
First off, the location was great, we were working close by at Boeing.
This old home in a historic neighborhood is so comfortable and convenient
This was a WONDERFUL introduction to Seattle!!We stayed at Catherine's place for six weeks while my husband did an internship in the area.
Best part was the  place was right on a few bus lines that take you straight to downtown Seattle.
The neighborhood is about 15ish minutes outside of Seattle by car.
The area wasn't the nicest, but that was our only complaint.
The bus stop was just down the road which made our trips to downtown easy, and the neighbourhood seemed fine.
Didn't spend a lot of time in th

Ah and the location is convenient, close to bus number 7 which brings you downtown in less than half and hour.
There is also a Walgreens nearby.
It was in a great location also with grocery and delivery near so you could relax at night and go into the city during the day with one easy bus ride.
Great friendly and helpful host, lovely artsy house, good location.
Would recommend his place to anyone staying in the area.
The apartment is comfortable and as advertised and the neighborhood is great.
Columbia City is a $20 Cab/Uber ride to Belltown/Downtown.
It is a safe, up and coming neighbourhood and we would definitely stay here again!
At the end of my month stay there AirBnb investigated and reimbursed me on my credit card for the night I had to leave near  1:00 AM to get some sleep.
Located in a great area, this spot is ideal for travelers who want access to the U District and surrounding neighborhoods.
The house is in a great area, and easy to locate.
Ravena is a beautiful, quiet neigh

Kristina's cottage has a great location & communicating with her was quick and easy, even though she was out of town.\n
The window next to shower made for an embarrassing morning shower as you hoped that you did not expose yourself to the next door neighbors.\n
The location is fabulous, convenient to lots of Seattle activities and sight-seeing.
This place is in a good location and fairly close to the city.
Perfect location too.
about so it was nice to come home to a quiet neighborhood.
The house is beautiful with lots of places to sleep (gorgeous kitchen) and only a 10 minute drive to downtown where we always park for free behind the stadiums.
The location was perfect and the price was right.
Also, Georgetown is just down the street, a kitschy, fun neighborhood with plenty of awesome bars, breweries, and restaurants.
There was continual airplane traffic overhead, but that's probably common most anywhere in the Seattle area.
The neighborhood was very noisy, which made it a bit hard to s

And the room, including public area was pretty neat.
The location is pretty close to I-5 High Way and the airport.
The suburb was just like any suburb back home in Australia.
There was a bus stop to Downtown Seattle a few metres from the house, which made it easier.\r\n\r\n
Catherine showed us around the place and to our room and gave us a wonderful recommendation on dinner options nearby.
The bus stop is just down the hill from the house near the Safeway (grocery store) and takes roughly 25-30 min to get to Pike Place Market/downtown area.
Uber and Lyft are both in the area and will run you $40-$50 one way.
If you are staying or visiting downtown or the south end of Seattle, I cannot recommend her place enough.
In a quiet neighborhood near bus lines & light rail.
The neighborhood has everything I needed within walking distance and good access to downtown and other neighborhoods.
Marin was a terrific host, so sweet and kind, she made the time to talk to you and give you great recommend

,num_of_sents_0.6,num_very_neg_0.6,num_neg_0.6,num_neut_0.6,num_pos_0.6,num_very_pos_0.6,num_sents
home_id,,,,,,,
264829,27,0,7,5,15,0,216
278192,6,0,1,1,3,1,68
286712,37,1,15,3,16,2,244
385438,3,0,1,0,1,1,27
571640,4,0,1,2,1,0,35
571651,0,0,0,0,0,0,24
606297,57,0,28,6,22,1,497
609610,19,0,8,2,9,0,77
611500,19,0,6,6,6,1,150


In [16]:
loc_score9_sents_sentiment_df = get_sentiment_summary(loc_score9_hs_df)
loc_score9_sents_sentiment_df

But so glad we did, her place was a really great location.
Its location is ideal (close to Capitol Hill, downtown, Lake Washington, and Puget Sound ferries) and the value is very good.
We walked to a grocery store nearby but used Uber for getting downtown.
There's a Trader Joe's nearby so we were able to stock up on essentials. \r\n\r\n
The location is ideal.
I'd definitely stay again!We had a wonderful two nights at Apryl's- it was near the end of our trip
A perfect escape and just a few minutes drive from the downtown area.
Great location, my girlfriend
Quick walk to the international district and downtown.
First, the location was great to access anywhere we wanted to go in Seattle.
Enjoyed a dip in her hot tub under the moon and had lovely walks around her neighborhood.
We really enjoyed walking near Lake Washington and eating in Columbia City.
Wonderful for a respite yet easy to re-route into downtown mode
The private garden cottage is a lovely place in a great Seattle neighborhood

Julie was helpful as she gave me all the locations I had asked for and the area space around the bungalow
The location was perfect!
This cottage is every bit as inviting, rejuvenating and peaceful as the pictures make it out to be - not to mention, the garden surrounding the property is beautiful!
The neighborhood itself offers plenty of opportunities to experience life as a local and when you're ready to leave the cottage access to Downtown Seattle and the sights is easy and comfortable to navigate. \r\n
I can't say enough how perfect this location is.
The downtown Columbia city district was so cute.
and I loved that there are white Christmas lights neatly hung near the ceiling, they were a great addition!
It is about a 15 minute walk to the lightrail and then another 15 minute or so ride into downtown as described... so not bad at all, but not REALLY central,
if that's what you're after (it is much more relaxing than downtown though).
The area around the house is cute, too - we did e

In terms of location, the Burke-Gilman trail is a few minutes walk and you can use this as an alternative route into Fremont, the main route being Leary Way, a busy four lane road which the deck partly overlooks.
Fremont is only ten minutes walk and buses 28 or 40 stop on Leary Way taking you downtown in 20-25 minutes.
The location is prime:  walking distance to all the cool stuff Fremont has to offer and with easy access to downtown, UW, and highways that take you to beautiful hiking and other outdoor activities all around the area.
The place was everything we hoped for, great location  near Fremont and Ballard, the hosts were great very helpful came to meet us a little early as we needed to get to an appointment soon after arriving.
Neighborhood is excellent and shops and restaurants are walking distance from the house.
The outdoor grill area and deck is awesome when it is not raining.
The only annoying thing is that the house is located off of a steep driveway.
We would definitely r

Just inside the back door is a common area with table and chairs, a refrigerator, microwave, and an individual serve coffee maker.
Close by, in walking distance, is a Whole Food Store and we were just a half block from a major bus line which if you take it you are in downtown Seattle in less than 10 minutes.
The neighbourhood was also safe.
He made us feel very comfortable, everything was very clean and the location is pretty close to downtown, you have everything what you need around that area.
Jean-Marc and his family could not be more welcoming and the surrounding flora and fauna - including his own couple puggles, chickens, and a bunny (not to mention the awesome hammock and swing) - make this a sort of paradise.
The place was very nice and clean and is in a great location.
Access to the common area was really nice.
We felt like we stayed in a fairy-tale....such a perfect location.
Easy access to downtown Seattle.
Jean-Marc and Maggie were both incredibly friendly and the location 

The location is awesome for any family with little ones.
This B&B is in the beautiful, safe & quiet neighborhood, and there's a Zoo and the lake nearby.  \n\n
The suite is in a great neighborhood, close to the park, zoo and restaurants.
Had we stayed in a hotel, we never would have gotten to know the neighborhood like we did; it was very homey, and more private than a hotel.
The neighborhood is awesome with easy access to Green Lake and Phinney Ridge restaurants.
the bathroom and kitchen/laundry area were probably my favorite.
We also loved having the kitchen area and made great use of the tea kettle.
Laurie and Gary had so many suggestions for the area and were very kind and accommodating.
The house was easy to find and in an excellent location.
The apartment was efficient, comfortable and the location was fabulous!
Great location.
The location is really ideal for almost everything in North Seattle and downtown is maybe 10 minutes.\r\n
Well-located and had everything we needed - inclu

...

It's not hip and trendy like Ballard, Green Lake or Fremont, but is conveniently located so all of the places mentioned above are within a short drive.
I would highly recommend this to anyone who is looking at staying in this neighborhood.
Lovely, quiet neighborhood, walking distance to tasty coffee/breakfast.
The neighborhood was quiet and street parking was never an issue.
We recommend a car rental if you want to see more of Seattle's different neighborhoods.
This is a great base if you want to stay near the Ballard/Fremont area.
Bed is fairly comfortable, and the sitting area is spacious for hanging out and getting ready.
The location is super convenient, with a large grocery store, a beer shop w/food trucks out front every day, restaurant options and a gas station a few blocks away.
I picked the location primarily because  it is just down the street from my daughter and grandchildren, but I am sure glad I did.
Very quiet neighborhood, didn't have trouble parking. \r\n
The studio wa

The apartment was just as described, a lovely clean modern space in a peaceful area, with easy street parking.
We live in Magnolia, so it was very convenient for us to have them staying in the neighborhood.
The yen location was about 15-20 min drive to most of the places we wanted to go to.
We were visiting family who live nearby, so the location was perfect for us.
Location was convenient, but parking complicated.
Located up the hill
It's right next to downtown ($5 uber ride away) and perfectly located.\r\n\r\n
Location is good too.
Public transport across the street was very handy to Downtown.
The location is perfect and so is the place.
The location could not be any better for downtown and waterfront (Pike's) and the loft has everything you need and then some!
It is 1.5 miles from Pike's market, walkable if not time constrained, or 8 minutes by Uber or Taxi.
Great location, near downtown and waterfront.
A little noisy since its by busy streetMack is easy to reach for checkin and que

I loved the urban, modern vibe that it offers and it was a perfect location to stay in the city of Seattle!
The location puts you within walking distance of downtown Madrona and just steps away from a 10 minute bus ride to downtown.
The apartment is very roomy, the kitchen is well-equipped, and the location is very convenient for the Capitol Hill neighborhood.
The bed was very comfortable and we both read in the main living area.
The neighborhood is adorable and their property is fabulous.
We enjoyed the location and felt like were coming home at night.
Easy cab ride to city center, downtown, etc., but far enough away to escape the city
Blair and Kate were very welcoming and accommodating, the apartment was massive and beautifully furnished/maintained, the location was in a cool neighborhood just outside of downtown, and they actually might have the best shower in the state of Washington.
Super accommodating host in a great neighborhood not too far from downtown.
The house is located o

Charming duplex, central and walkable to many great parts of Seattle.
It was pretty spacious and not far from downtown Seattle.
The location was great for us.
I am actually from the neighbourhood
House is walking distance to Portage Bay, a great brunch spot and Wallingford is very walkable.
This is my 5th AirB&B, in urban and rural areas, so I feel that I have a decent basis of comparison.
It is near some great restaurants and a great bike/running trail, which was nice. \r\n
One of my favorite Seattle neighborhoods.
The location was convenient for us to explore Seattle by car or Uber: dumplings at Din Tai Fung, Seahawks game, Marination Station in West Seattle, Living Computer Museum, etc.
We enjoyed the neighborhood and the access to downtown Seattle.
The apartment was clean, spacious and in a good location ( close to Burke-Gilman trail, UW & bus connections).
The neighborhood was very convenient to Wallingford and Seattle.
Great location close to the city, shops and food.
The locatio

,num_of_sents_0.6,num_very_neg_0.6,num_neg_0.6,num_neut_0.6,num_pos_0.6,num_very_pos_0.6,num_sents
home_id,,,,,,,
202174,10,0,2,1,6,1,127
215882,142,0,35,11,82,14,964
219532,45,1,13,6,23,2,237
749758,9,0,3,0,6,0,50
762029,125,0,38,7,71,9,979
788146,15,0,4,0,7,4,137
1029680,1,0,0,1,0,0,16
1133329,85,1,25,4,44,11,468
1171574,141,1,34,21,76,9,836


In [17]:
loc_score10_sents_sentiment_df = get_sentiment_summary(loc_score10_hs_df)
loc_score10_sents_sentiment_df

We loved the the neighborhood and the surroundings and enjoyed the easy walk down to the park.
Great location.
Walllingford is close to downtown, the Burke-Gilman trail, lots of shops, etc.
Her house is centrally located and stocked with all the amenities and comforts one could dream of (hot tub included).
The location is excellent.
It is walking distance to several great neighborhood coffee shops and to 45th street where everything is happening.
It is a great neighborhood for walking and you feel very safe.
Great location.
Everyone enjoyed the lovely garden, neighborhood and of course the classic turn of the \r\ncentury home. \r\n
I loved staying in the Wallingford / Fremont area plus the bus is 2 minute walk and a 15 minute ride downtown to the major attractions of Seattle.  \r\n\r\n
It is a quiet location central to great shopping, bus lines and wonderful restaurants.
It is easy to catch a bus downtown and to other locations in Seattle.
Definitely recommended to live here if you're 

Our trip was a bit of a whirlwind—we needed to stay somewhere in the neighborhood due to a series of family events over a holiday weekend, and Kim's house would've been perfect on location alone
(it's in a good spot to get to almost anywhere in the city quickly by car, yet you're tucked into a very quiet, safe neighborhood).
There's plenty of nearby restaurants and shops.\r\n\r\n
As such, I can't comment on how access to public transit is from this location.
The streets in this neighborhood are narrow and generally parked-in on both sides; if your rental car is a giant SUV or something, it may be a challenge getting in and out.\r\n\r\n3.
It is in a great location, with wonderful shops, restaurants, and coffee shops within walking distance.
The apartment is also a very easy drive downtown - we were to the space needle, waterfront, etc within 10 minutes.
Both Kim and Stewart have a wealth of knowledge on things to do, places to eat, etc in the area and in Seattle as well, which we were t

The listing description was accurate, in a good residential location overlooking the city of Seattle (west side of the building), near Broadway, with lots of excellent places to eat, and a busy Starbucks down the street.
The apartment is super close to core area in Capital Hill.
Within a walkable distance, you can visit Broadway, downtown.\r\n
Great location.
Great neighborhood and great apartment.
Just a note though that this is not an elevator building and this particular unit is located on the third floor. \r\n\r\n
It's a great location and very close to great restaurants.
It is centrally located, close to everything you would need or want, clean, updated, and in a great, safe neighborhood.
This location was perfect, and we truly enjoyed our stay.
I would highly recommend visiting Seattle, and utilizing this location.
So close to downtown too :)
I would definitely stay there againWonderful updated apartment and location, close to many places, bars, coffee shops, supermarket and rest

Great location
and I could walk to downtown and space needle within 20 minutes.
Their home is warm and lovely and in an ideal location.
I loved the neighborhood and the convenience to downtown.
Parking was difficult being in the Capitol Hill neighborhood
The location is quiet, but very close to the heart of Capitol Hill restaurants and shops.
The location is perfect.
The location is perfect, close to where you really want to be.
Very clean room and convenient to downtown.
The location was great.
Also on several bus routes from downtown, great for everywhere.
Chad and Ruben are super considerate hosts with a charming, clean, and conveniently-located house.
Great location for coffee and brews.
The place is centrally located and tidy and their dog is gorgeous.
The location and room we were provided was PERFECT.
There is a lot of construction going on around the Capitol Hill district, and many streets have sections blocked off.\r\n\r\n
The location is convenient and I was able to get what 

The neighborhood was wonderful for walking, dining, people-watching, house-looking, and some shopping.
The neighborhood is also quite nice and is in a great location.
The neighborhood is good and not too far from Downtown Seattle for either walking or buses.
Also an easy walk to downtown and even the Space Needle if your feeling frisky.
2 other locations
The neighborhood was also nice, quite calm in their street but with more life (bars and restaurants) only some steps away.
The location is great and walking distance to all the great places in Capitol Hill.
Tons of Car2Gos around as well :)Super comfortable, conveniently located, and easily accessible room!
Great location.
Great location, smooth check-in and check-out and a good value.\r\n
It was located an easy walk to  many downtown destinations.
The apt is perfectly located in the heart of downtown Seattle.
We loved our stay there and would definitely recommend!Excellent communication, location and experience.
The parking area can b

...

The neighborhood itself is called Alaska Junction.
Nice and clean room!Very friendly host!Convenient location and quite neighborhood!everything what u
I would def go back to stay in the lovely neighborhood for a longer stay...
The neighborhood is pretty to walk through and the public transportation is convenient and very fast to downtown.
Her house is on a quiet street, perfectly located between Alki Beach, West Seattle Junction and Admiral.
The room was comfortable and it's located in a nice quiet neighborhood.
It is conveniently located in Capital Hill, with various bars and grocery stores within walking distance.
I would rent this location for a longer period if I could.
Overall, this was a terrific location to stay at during my month in Seattle.
I primarily chose this site due to its location being in such close proximity to where I would be working (a little over a mile away).
This room specifically is it's own floor with a private deck that provides gorgeous views of the lake, po

It was a good location for us as we explored the city of Seattle!
Completely as we expected, a nice flat with great location.\r\n\r\n
The location is excellent, close to everything you need.
The location is in a nice neighborhood within walking distances to bars and restaurants and super close to public transport.
She was always available to answer our questions about the area.
We walked around the neighbourhood, had great chats, got bites to eat and drinks to...
The Fremont neighborhood is quaint and convenient, with a bus line that takes you right to the Seattle Center and downtown.
Very clean and awsome location.
She gave us a lot of suggestions as to what was in the area and how to get the best views of fireworks for 4th of July.
The location was clean and the bed was comfortable.
Also a great location,  easy walking distance to resturaunt, bars, the troll and a bus line stops right across the street to take you downtown.
You couldn't ask for a better location to experience Seattle

Great location.
really great location!
We enjoyed the neighborhood and all of the wonderful shops/restaurants nearby.\r\n\r\n
The neighborhood was just what we were looking for and we had a great time!We only talked to Carolyn and James once on our trip and they were very friendly.
There was a ton to do in the area even though the house was tucked away on a quiet residential street.
Their place is in a great residential area of Wallingford.
Coffee shops, a supermarket, restaurants- all are only a few blocks away, but nonetheless at night the neighborhood is quiet as a mouse.
The neighborhood is excellent with many restaurants, bars, coffee shops, and a grocery store all within a few blocks.
Their home is in an interesting and pedestrian friendly neighborhood.
Thanks again!This apartment is in a cute neighborhood.
Carolyn & James made sure to contact me prior to check-in regarding key location.
We had everything we needed, and the location is fantastic.
Just a short uber ride away from 

The neighborhood is quite and beautiful, not hard to find, and close to some quaint little neighborhood centers.
The location is very convenient to hiking/biking trails, the U District, and to the highway.
The place is immaculate and there are special touches throughout from cozy socks to wear, area magazines, and beautiful rugs in the bedroom and hallway.
This location is close to parks, the University of Washington, the Burke-Gilman Trail, and the University Village for shopping.
Their unit offered plenty of privacy, was located conveniently to small shops, and within walking distance of fabulous views of Mt. Rainier.
A great place to relax away from my usual downtown haunts.
The location was perfect, close to Magnusson Park, the district and many great restaurants.
I lived near this neighborhood for many years in Seattle and Judith's beautiful house and hospitality made me feel at home again.
Although off the beaten path, this home is in a great part of Seattle and can really give y

,num_of_sents_0.6,num_very_neg_0.6,num_neg_0.6,num_neut_0.6,num_pos_0.6,num_very_pos_0.6,num_sents
home_id,,,,,,,
9596,31,0,8,2,19,2,173
20928,45,0,10,1,28,6,212
254340,76,1,23,7,39,6,462
387078,58,0,13,8,30,7,271
557126,175,2,43,14,101,15,917
934123,11,0,1,1,9,0,34
935432,6,0,1,1,4,0,34
1030411,6,0,1,0,5,0,21
1167507,98,0,37,9,42,10,396


In [18]:
# clean_score7_sents_sentiment_df = pd.concat([clean_score7_hs_df, clean_score7_sents_sentiment_df], axis=1).drop(['num_sents'], axis=1)
# clean_score8_sents_sentiment_df = pd.concat([clean_score8_hs_df, clean_score8_sents_sentiment_df], axis=1).drop(['num_sents'], axis=1)
# clean_score9_sents_sentiment_df = pd.concat([clean_score9_hs_df, clean_score9_sents_sentiment_df], axis=1).drop(['num_sents'], axis=1)
# clean_score10_sents_sentiment_df = pd.concat([clean_score10_hs_df, clean_score10_sents_sentiment_df], axis=1).drop(['num_sents'], axis=1)

# loc_score7_sents_sentiment_df = pd.concat([loc_score7_hs_df, loc_score7_sents_sentiment_df], axis=1).drop(['num_sents'], axis=1)
# loc_score8_sents_sentiment_df = pd.concat([loc_score8_hs_df, loc_score8_sents_sentiment_df], axis=1).drop(['num_sents'], axis=1)
# loc_score9_sents_sentiment_df = pd.concat([loc_score9_hs_df, loc_score9_sents_sentiment_df], axis=1).drop(['num_sents'], axis=1)
# loc_score10_sents_sentiment_df = pd.concat([loc_score10_hs_df, loc_score10_sents_sentiment_df], axis=1).drop(['num_sents'], axis=1)

In [19]:
clean_score7_sents_sentiment_df.to_csv('./Data/sentiment_counts/clean_sents_stm_df_7.csv', sep='\t', encoding='utf-8')
clean_score8_sents_sentiment_df.to_csv('./Data/sentiment_counts/clean_sents_stm_df_8.csv', sep='\t', encoding='utf-8')
clean_score9_sents_sentiment_df.to_csv('./Data/sentiment_counts/clean_sents_stm_df_9.csv', sep='\t', encoding='utf-8')
clean_score10_sents_sentiment_df.to_csv('./Data/sentiment_counts/clean_sents_stm_df_10.csv', sep='\t', encoding='utf-8')
loc_score7_sents_sentiment_df.to_csv('./Data/sentiment_counts/loc_sents_stm_df_7.csv', sep='\t', encoding='utf-8')
loc_score8_sents_sentiment_df.to_csv('./Data/sentiment_counts/loc_sents_stm_df_8.csv', sep='\t', encoding='utf-8')
loc_score9_sents_sentiment_df.to_csv('./Data/sentiment_counts/loc_sents_stm_df_9.csv', sep='\t', encoding='utf-8')
loc_score10_sents_sentiment_df.to_csv('./Data/sentiment_counts/loc_sents_stm_df_10.csv', sep='\t', encoding='utf-8')